# Convolutional Neural Network

### Importing the libraries

In [ ]:
# Import necessary libraries
import tensorflow as tf  # TensorFlow for deep learning
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Keras module for image preprocessing

In [ ]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,         # Normalize pixel values to a range of [0,1] (originally [0,255])
        shear_range=0.2,        # Apply random shearing transformation to the images
        zoom_range=0.2,         # Apply random zooming to images
        horizontal_flip=True,   # Enable random horizontal flipping of images
        fill_mode='nearest'     # Fill in missing pixels after transformations with the nearest pixel value
)

train_generator = train_datagen.flow_from_directory(
        'data/train',        # Path to the directory containing training images
        target_size=(64, 64),  # Resize all images to 150x150 pixels
        batch_size=32,       # Number of images to be processed in each batch
        class_mode='binary'  # Binary classification (e.g., cats vs. dogs), suitable for binary_crossentropy loss
)


### Preprocessing the Test set

In [ ]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)
# this is a similar generator, for validation data
test_set = test_datagen.flow_from_directory(
        'data/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
# Adding a Convolutional Layer to the CNN
cnn.add(tf.keras.layers.Conv2D(
    filters=32,          # Number of filters (kernels), defining how many feature maps will be learned
    kernel_size=3,       # Size of each filter (3x3 in this case), determining the receptive field
    activation='relu',   # Activation function used to introduce non-linearity and prevent vanishing gradients
    input_shape=[64, 64, 3]  # Shape of the input image: 64x64 pixels with 3 color channels (RGB)
))

### Step 2 - Pooling

In [ ]:
# Adding a Max Pooling Layer to the CNN
cnn.add(tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2),  # Size of the pooling window (2x2), reducing spatial dimensions by taking the max value in each window
    strides=2          # Step size for moving the pooling window, effectively downsampling the feature map
))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(
    filters=32,          # Number of filters (kernels), defining how many feature maps will be learned
    kernel_size=3,       # Size of each filter (3x3 in this case), determining the receptive field
    activation='relu',   # Activation function used to introduce non-linearity and prevent vanishing gradients
    #input_shape=[64, 64, 3]  # Shape of the input image: 64x64 pixels with 3 color channels (RGB) only for first layer
))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))

### Step 3 - Flattening

In [ ]:
# Flattening the feature maps into a 1D vector
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
# Adding a Fully Connected (Dense) Layer to the CNN
cnn.add(tf.keras.layers.Dense(
    units=128,        # Number of neurons in this layer, controlling the learning capacity
    activation='relu' # Activation function, introducing non-linearity and improving learning of complex patterns
))

### Step 5 - Output Layer

In [ ]:
# Adding the Output Layer to the CNN
cnn.add(tf.keras.layers.Dense(
    units=1,         # Number of output neurons; 1 neuron for binary classification (e.g., cat vs. dog)
    activation='sigmoid'  # Sigmoid activation function to output a probability between 0 and 1
))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
# Compiling the CNN model
cnn.compile(
    optimizer='adam',              # Optimizer used to update weights efficiently (Adam is widely used for deep learning)
    loss='binary_crossentropy',    # Loss function for binary classification (use 'categorical_crossentropy' for multi-class)
    metrics=['accuracy']           # Metric to evaluate model performance (accuracy is commonly used for classification)
)

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# Defining an Early Stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',          # Monitors the validation loss during training
    patience=7,                  # Stops training if validation loss doesn't improve for 5 consecutive epochs
    restore_best_weights=True    # Restores the model's best weights before stopping
)

In [ ]:
# Training the CNN model
cnn.fit(
    x=train_generator,        # Training dataset (augmented images from the data generator)
    validation_data=test_set, # Validation dataset to monitor model performance
    epochs=25,                # Number of complete passes over the entire training dataset
    callbacks=[early_stop]    # Uses EarlyStopping to stop training if no improvement in validation loss
)

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Loading and preprocessing the test image
test_image = image.load_img(
    'data/single_prediction/cat_or_dog_1.jpg',  # Path to the image to be tested
    target_size=(64, 64)  # Resizing the image to match the CNN input shape
)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Making a prediction using the trained CNN
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

print(prediction)